# Projeto 1 - Ciência dos Dados

Nome: Carlos Simodo

Nome: Luiz Fernando da Silva Borges

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Obtenção dos tweets

## Preparando o ambiente no jupyter:

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [1]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @hoxie_reader ]***

In [14]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas para construção da base de dados:

### Escolha de um produto e coleta das mensagens


In [32]:
#Produto escolhido:
produto = 'Lula'

#Quantidade mínima de mensagens capturadas:
n = 3000

#Quantidade mínima de mensagens para a base de treinamento:
t = 668

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [33]:
#Cria um objeto para a captura
api = tweepy.API(auth, wait_on_rate_limit=True)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break


In [42]:
#Embaralhando as mensagens para reduzir um possível viés temporal de obtenção 

shuffle(msgs)

print(len(msgs))

1002


In [43]:
msgs = list(set(list(msgs)))

len(msgs)

1002

Uma vez que a API capturou 2000 tweets, decidimos prosseguir com 1002 tweets no total. 


In [44]:
lista_cortada = msgs[:1002]
msgs = lista_cortada

len(msgs)

1002

In [53]:
#Verifica se o arquivso não existem para não substituir um conjunto pronto
if not os.path.isfile('./mensagens_1.xlsx') and os.path.isfile('./mensagens_2.xlsx'):
    
    #pega a quantidade de metade das mensagens escolhidas (deve ser par)
    metade_mensagens = int(len(msgs)/2)
    
    #Abre o arquivo para escrita
    writer1 = pd.ExcelWriter("mensagens_1A.xlsx")
    writer2 = pd.ExcelWriter("mensagens_2A.xlsx")
    
    #divide o conjunto de mensagens totais em duas planilhas, pois trata-se de um projeto em dupla
    dft1 = pd.DataFrame({"Tweets": pd.Series(msgs[:metade_mensagens])})
    dft2 = pd.DataFrame({"Tweets": pd.Series(msgs[metade_mensagens:])})
    
    #converte os data frames para arquivos .xlsx
    dft1.to_excel(excel_writer = writer1, index = False)
    writer1.save()
    
    dft2.to_excel(excel_writer = writer2, index = False)
    writer2.save()
    

In [3]:
#junta os dois arquivos excel em um unico
df_mensagens_etiquetadas_1 = pd.read_excel('tabela_etiquetada_1.xlsx') 
df_mensagens_etiquetadas_2 = pd.read_excel('tabela_etiquetada_2.xlsx') 

#a coluna foi nomeada manualmente de 'Tweets'
valores1 = df_mensagens_etiquetadas_1[['Tweets','Etiquetas']]
valores2 = df_mensagens_etiquetadas_2[['Tweets','Etiquetas']]

df_tabela_etiquetada = pd.concat([valores1, valores2])
#df_tabela_etiquetada
#print(len(df_tabela_etiquetada['Etiquetas']))

In [5]:
#verifica se o numero de cada tipo etiquetas ultrapassa ou alcanca o minimo necessario

    #valor minimo de cada tipo de etiqueta
valor_minimo=252

    #funcao responsavel pela verificacao
def maior_que_250(df):
    #define resposta padrao como verdadeiro
    avancar=True
    #conta o numero de vezes que cada etiqueta se repete e coloca numa lista
    distribuicao = df['Etiquetas'].value_counts()
    lista_dos_numeros_de_cada_tipo_de_etiqueta = distribuicao.to_numpy()
    print(lista_dos_numeros_de_cada_tipo_de_etiqueta)
    #caso algum dos numero nao passe, a resposta vai se tornar falsa
    for valor in lista_dos_numeros_de_cada_tipo_de_etiqueta:
        if valor>=valor_minimo:
            pass
        else:
            print(valor)
            avancar=False
    return avancar

print(maior_que_250(df_tabela_etiquetada))

[579 267 156]
156
False


In [6]:
def mesmo_numero(df):
    #distribuicao de etiquetas
    distribuicao = df['Etiquetas'].value_counts()

    #obtem a contagem de cada etiqueta
    lista_dos_numeros_de_cada_tipo_de_etiqueta = distribuicao.to_numpy()

    #descobre o numero de aparicoes da etiqueta menos frequente
    menor_numero=min(lista_dos_numeros_de_cada_tipo_de_etiqueta)

    #cria filtros
    filtro_p = df['Etiquetas'] == 1
    filtro_n = df['Etiquetas'] == 2
    filtro_i = df['Etiquetas'] == 3

    #filtra o dataframe
    df_p = df.loc[filtro_p]
    df_n = df.loc[filtro_n]
    df_i = df.loc[filtro_i]
    
    #reorganiza os dataframes
    df_p = df_p.sample(frac = 1) 
    df_n = df_n.sample(frac = 1) 
    df_i = df_i.sample(frac = 1) 
    
    #slicing
    lista_p = df_p.iloc[0:menor_numero,]
    lista_n = df_n.iloc[0:menor_numero,]
    lista_i = df_i.iloc[0:menor_numero,]
    
    lista_data_frames=[lista_p,lista_n,lista_i]
    return lista_data_frames

lista_data_frames = mesmo_numero(df_tabela_etiquetada)
lista_data_frames

[                                                Tweets  Etiquetas
 399  @uol eu sou pt mas vai depender do candidato,t...          1
 56   rt @leonicemariana1: pt confirma candidatura d...          1
 401  os insensíveis desse desgoverno, não ficam cho...          1
 302  @evilzio5 @samiabomfim @geancarlobc @damaresal...          1
 213  @agencialupa @redescordiais esse tesão que os ...          1
 ..                                                 ...        ...
 413  rt @reportersalles: o líder do governo na câma...          1
 153  @veramagalhaes @gabilotta @fgv @felipesalto @i...          1
 372  @oiiuiz verdade! não se pode fazer com lula o ...          1
 115  @inst_lula @lulaoficial #voltalula e salve o b...          1
 287  rt @henr1quefragoso: a propósito da correta fa...          1
 
 [156 rows x 2 columns],
                                                 Tweets  Etiquetas
 31         rt @carlos38305595: faltou : lula é ladrão.          2
 124  rt @viniciuscfp82: arrasou! m

In [7]:
#separar em teste e treino, tendo a mesma razao de porcentagem de cada tipo de etiqueta
import numpy as np
def separa_teste_treino(lista):
    #define os termos da lista
    df_p=lista[0]
    df_n=lista[1]
    df_i=lista[2]
    
    #define a divisao
    divisao=1/3
    
    #separa os dataframes em dois novos dataframes
    df_p_teste, df_p_treino= np.split(df_p, [int(divisao*len(df_p))])
    df_n_teste, df_n_treino= np.split(df_n, [int(divisao*len(df_n))])
    df_i_teste, df_i_treino= np.split(df_i, [int(divisao*len(df_i))])
    
    #junta os dfs parcias nos dfs de teste e treino
    df_teste = pd.concat([df_p_teste, df_n_teste,df_i_teste],axis=0)
    df_treino = pd.concat([df_p_treino, df_n_treino,df_i_treino],axis=0)
    
    #reorganiza os dfs
    #df_teste = df_teste.sample(frac = 1)
    #df_treino = df_treino.sample(frac = 1)

    
    #junta os dfs em uma lista
    novos_dfs = [df_teste,df_treino]
    return novos_dfs

#chama a funcao
novos_dfs=separa_teste_treino(lista_data_frames)
df_teste = novos_dfs[0]
df_treino = novos_dfs[1]
print(df_teste)
print(df_treino)

#conta o numero de vezes que cada etiqueta se repete e coloca numa lista
distribuicao = df_teste['Etiquetas'].value_counts()
lista_dos_numeros_de_cada_tipo_de_etiqueta = distribuicao.to_numpy()
print(lista_dos_numeros_de_cada_tipo_de_etiqueta)

distribuicao = df_treino['Etiquetas'].value_counts()
lista_dos_numeros_de_cada_tipo_de_etiqueta = distribuicao.to_numpy()
print(lista_dos_numeros_de_cada_tipo_de_etiqueta)

#testes
#separa a funcao em df
#df_p_teste = listas[0]
#df_p_treino = listas[1]
#df_n_teste = listas[2]
#df_n_treino = listas[3]
#df_i_teste = listas[4]
#df_i_treino = listas[5]

#print(len(df_p_teste.index))
#print(len(df_p_treino.index))
#print(len(df_n_teste.index))
#print(len(df_n_treino.index))
#print(len(df_i_teste.index))
#print(len(df_i_treino.index))

                                                Tweets  Etiquetas
399  @uol eu sou pt mas vai depender do candidato,t...          1
56   rt @leonicemariana1: pt confirma candidatura d...          1
401  os insensíveis desse desgoverno, não ficam cho...          1
302  @evilzio5 @samiabomfim @geancarlobc @damaresal...          1
213  @agencialupa @redescordiais esse tesão que os ...          1
..                                                 ...        ...
325              @luizpraieiro @rodrigomaia foi o lula          3
433  rt @macunaimaz: @marialcab @trotzer2 @ocyarada...          3
333  rt @jack_onil: @brumarquezine n tenho político...          3
65   rt @fcomorais: @trotzer2 @ocyaradamasio @profe...          3
221  @suzete040428071 suzy o que podemos fazer com ...          3

[156 rows x 2 columns]
                                                Tweets  Etiquetas
20   rt @elisaraujo2010: lula falou sobre questão d...          1
361  lava jato inventa nova denúncia contra lula ap.

Salvando os dados em uma planilha Excel:

In [95]:
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))
    
    df_treinamento = df_tabela_etiquetada[:t]
    df_treinamento.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = True) 
    
    df_teste = df_tabela_etiquetada[t:]
    df_teste.to_excel(excel_writer = writer, sheet_name = 'Teste', index = True) 
    
    writer.save()
    

In [ ]:
##########################################################################################

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

**Importante: Caso classifique um percentual pequeno de tweets relevantes ou de não relevantes, deve voltar a este notebook e coletar mais tweets diferentes do produto escolhido pelo grupo.**